<a href="https://colab.research.google.com/github/jihun0423/Baseball-pitcher-predict/blob/main/baseball.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 이번에 알아보고 싶은 것

야구에서 투수의 성적을 예측할 때에는, 여러가지 변수가 있다.
타자가 투수의 공을 얼마나 잘 치는가, 투수가 속한 팀의 수비 수준은 어떠한가,
투수가 투구하는 구장 환경은 어떠한가 등등 투수가 공을 던지는 능력과는 무관하다고 볼 수 있는 변수들이 매우 많다.

나는 여기서 수비와 운이라는 변수를 제외한 투수의 성적 예측하고자 한다. 따라서, 최대한 투수의 능력과 관련이 있는 변수들만을 사용해 예측을 할 예정이다.

이를테면, 투수가 던지는 공의 구속, 변화 정도, 스핀 수 등은 투수의 순수 능력이다.
다만, 위의 지표만 이용해서 분석을 할 경우, 구속이 빠른 투수가 예측 성적이 좋게 나올 확률이 높다. 따라서 제구력과 관련된 지표들을 여럿 넣었다. 

BABIP 이론에 따른다면, 타자가 배트에 공을 맞추었을때 나타나는 상황(아웃, 안타 등)은 어느 투수이던 간에 확률이 똑같다고 한다. 인플레이 타구의 결과는 투수와 수비수 모두의 책임이다. 따라서 인플레이 타구에 대해 투수의 책임이 있는 부분만을 다루는 스탯이 필요하며 이것이 바로 이번 예측에 사용할 지표인 xwoba 이다.



# 임포트

In [1]:
# encoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler


from sklearn.preprocessing import MinMaxScaler

# machine learning models
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from xgboost import XGBRegressor
from sklearn.ensemble import ExtraTreesRegressor

!pip install pycaret
from pycaret.regression import *

# pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

!pip install skorch

from skorch import NeuralNetRegressor
from sklearn.base import BaseEstimator, TransformerMixin

import pandas as pd
import numpy as np

from sklearn.model_selection import GridSearchCV, KFold
from sklearn import set_config

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F

import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.8/105.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 8.0 MB/s eta 0:00:00


In [2]:
! pip install pandasql   # 처음에만 실행

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=92a8c032fc6a5021146b09787186ef1eb1a12a716a6cf5112b768aceb321dabf
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql


In [3]:
! pip install SQLAlchemy==1.4.46   # 처음에만 실행

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.9 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.10
    Uninstalling SQLAlchemy-2.0.10:
      Successfully uninstalled SQLAlchemy-2.0.10


# 데이터 출처

데이터는 BASEBALL-SAVANT라는 평소 매우 애용하고 있는 야구 통계 사이트에서 가져왔다. 자신이 원하는 컬럼만을 선택 할 수 있어, 전처리 과정이 편해진다는 장점이 있다.

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
cd/content/gdrive/MyDrive

/content/gdrive/MyDrive


In [6]:
df=pd.read_csv("stats (4).csv")

In [56]:
test=pd.read_csv("stats (6).csv")

# 전처리

In [7]:
df.isnull().sum()

last_name                   0
 first_name                 0
player_id                   0
year                        0
p_walk                      0
xwoba                       0
oz_swing_percent            0
oz_swing_miss_percent       0
oz_contact_percent          0
out_zone_swing_miss         0
out_zone_swing              0
out_zone_percent            0
out_zone                    0
pitch_count                 0
iz_contact_percent          0
in_zone_swing_miss          0
in_zone_swing               0
in_zone_percent             0
in_zone                     0
edge_percent                0
edge                        0
whiff_percent               0
n_fastball_formatted        0
fastball_avg_speed          0
fastball_avg_spin           0
fastball_avg_break          0
n_breaking_formatted       35
breaking_avg_speed         35
breaking_avg_spin          36
breaking_avg_break         35
n_offspeed_formatted      567
offspeed_avg_speed        567
offspeed_avg_spin         567
offspeed_a

결측치가 존재. 커브 같은 브레이킹볼이나, 체인지업류의 오프스피드 피치를 던지지 않는 투수들이 있어 생긴 결측치이다.

이 투수들은 저 구종들을 던지지 않으므로 구속, 회전수, 변화를 전부 0으로 둔 뒤,
브레이킹볼을 던지는지, 오프스피드를 던지는지 여부를 라벨 인코딩을 할 예정이다.

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4198 entries, 0 to 4197
Data columns (total 35 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   last_name              4198 non-null   object 
 1    first_name            4198 non-null   object 
 2   player_id              4198 non-null   int64  
 3   year                   4198 non-null   int64  
 4   p_walk                 4198 non-null   int64  
 5   xwoba                  4198 non-null   float64
 6   oz_swing_percent       4198 non-null   float64
 7   oz_swing_miss_percent  4198 non-null   float64
 8   oz_contact_percent     4198 non-null   float64
 9   out_zone_swing_miss    4198 non-null   int64  
 10  out_zone_swing         4198 non-null   int64  
 11  out_zone_percent       4198 non-null   float64
 12  out_zone               4198 non-null   int64  
 13  pitch_count            4198 non-null   int64  
 14  iz_contact_percent     4198 non-null   float64
 15  in_z

In [9]:
df.head(3)

,last_name,first_name,player_id,year,p_walk,xwoba,oz_swing_percent,oz_swing_miss_percent,oz_contact_percent,out_zone_swing_miss,...,fastball_avg_break,n_breaking_formatted,breaking_avg_speed,breaking_avg_spin,breaking_avg_break,n_offspeed_formatted,offspeed_avg_speed,offspeed_avg_spin,offspeed_avg_break,Unnamed: 34
0,Colon,Bartolo,112526,2016,32,0.318,30.2,22.4,75.9,91,...,17.4,6.3,82.5,2282.0,7.9,4.3,81.4,1683.0,16.2,NaN
1,Grilli,Jason,276351,2016,32,0.305,28.4,49.1,50.3,80,...,20.5,35.7,81.1,2605.0,13.9,2.4,84.9,2134.0,19.9,NaN
2,Benoit,Joaquin,276542,2016,24,0.294,34.7,40.9,57.3,67,...,19.1,19.4,87.2,2422.0,6.8,26.1,84.5,1374.0,13.7,NaN


## 결측치 제거를 위한 피쳐 생성

In [10]:
df['throws_breaking'] = 1
df.loc[df['n_breaking_formatted'].isnull()==1,'throws_breaking'] =0             # 브레이킹볼을 던지는지 여부에 따른 인코딩

In [60]:
test['throws_breaking'] = 1
test.loc[test['breaking_avg_spin'].isnull()==1,'throws_breaking'] =0   

In [11]:
df['throws_offspeed'] = 1
df.loc[df['n_offspeed_formatted'].isnull()==1,'throws_offspeed'] =0             # 오프스피드볼을 던지는지 여부에 따른 인코딩

In [61]:
test['throws_offspeed'] = 1
test.loc[test['offspeed_avg_spin'].isnull()==1,'throws_offspeed'] =0 

## 결측치 제거

In [34]:
df = df.fillna(0) # 결측치들을 전부 0으로 변경.

In [62]:
test = test.fillna(0)

## 불필요한 컬럼 제거

In [17]:
df.columns

Index(['last_name', ' first_name', 'player_id', 'year', 'p_walk', 'xwoba',
       'oz_swing_percent', 'oz_swing_miss_percent', 'oz_contact_percent',
       'out_zone_swing_miss', 'out_zone_swing', 'out_zone_percent', 'out_zone',
       'pitch_count', 'iz_contact_percent', 'in_zone_swing_miss',
       'in_zone_swing', 'in_zone_percent', 'in_zone', 'edge_percent', 'edge',
       'whiff_percent', 'n_fastball_formatted', 'fastball_avg_speed',
       'fastball_avg_spin', 'fastball_avg_break', 'n_breaking_formatted',
       'breaking_avg_speed', 'breaking_avg_spin', 'breaking_avg_break',
       'n_offspeed_formatted', 'offspeed_avg_speed', 'offspeed_avg_spin',
       'offspeed_avg_break', 'throws_breaking', 'throws_offspeed'],
      dtype='object')

In [22]:
df= df.drop(columns=['p_walk','n_fastball_formatted','n_breaking_formatted','n_offspeed_formatted','Unnamed: 34','pitch_count'])     
test=test.drop(columns=['Unnamed: 30','p_walk'])
  # 불필요해 보이는 컬럼들 제거

## 데이터 타입 변경

In [23]:
df['year'] = df['year'].astype('category')
df['player_id'] = df['player_id'].astype('category')
df['throws_breaking'] = df['throws_breaking'].astype('category')
df['throws_offspeed'] = df['throws_offspeed'].astype('category')                    # 데이터들의 타입을 바꿔주었다.
test['throws_breaking'] = test['throws_breaking'].astype('category')
test['throws_offspeed'] = test['throws_offspeed'].astype('category')     

In [27]:
df.corr()              # 타겟인 xwoba와 피쳐들간의 상관관계가 꽤 있어보인다. 

,xwoba,oz_swing_percent,oz_swing_miss_percent,oz_contact_percent,out_zone_swing_miss,out_zone_swing,out_zone_percent,out_zone,pitch_count,iz_contact_percent,...,whiff_percent,fastball_avg_speed,fastball_avg_spin,fastball_avg_break,breaking_avg_speed,breaking_avg_spin,breaking_avg_break,offspeed_avg_speed,offspeed_avg_spin,offspeed_avg_break
xwoba,1.000000,-0.407038,-0.349433,0.348366,-0.277688,-0.195538,0.128809,-0.143167,-0.153254,0.449539,...,-0.497745,-0.219567,-0.186529,-0.022629,-0.120639,-0.090632,-0.049689,-0.148524,-0.085356,-0.014104
oz_swing_percent,-0.407038,1.000000,0.149919,-0.149253,0.329273,0.296584,0.140452,0.163625,0.154101,-0.162983,...,0.319431,0.000618,0.068179,0.001728,0.099252,-0.020691,-0.120847,0.003322,0.031540,-0.011596
oz_swing_miss_percent,-0.349433,0.149919,1.000000,-0.995176,0.226774,-0.004216,0.139687,-0.032810,-0.040254,-0.386144,...,0.805937,0.385993,0.271188,-0.009789,0.318263,0.181020,-0.088050,0.325518,0.009247,-0.088131
oz_contact_percent,0.348366,-0.149253,-0.995176,1.000000,-0.231404,-0.002196,-0.140408,0.025390,0.032916,0.383450,...,-0.799880,-0.381746,-0.266079,0.012309,-0.315860,-0.180064,0.087760,-0.324234,-0.011903,0.088023
out_zone_swing_miss,-0.277688,0.329273,0.226774,-0.231404,1.000000,0.953075,0.045514,0.924077,0.915949,-0.094503,...,0.221210,0.053503,0.081758,0.007919,0.035142,0.064661,0.033148,0.076150,0.010852,0.006998
out_zone_swing,-0.195538,0.296584,-0.004216,-0.002196,0.953075,1.000000,0.020732,0.983660,0.976713,0.011391,...,0.022205,-0.055310,0.008600,0.007234,-0.057229,0.021909,0.064450,-0.008313,0.012273,0.032223
out_zone_percent,0.128809,0.140452,0.139687,-0.140408,0.045514,0.020732,1.000000,0.000477,-0.076730,0.049473,...,0.198368,-0.077431,-0.020692,-0.075169,0.021128,0.043898,-0.066029,-0.010227,-0.015555,-0.053426
out_zone,-0.143167,0.163625,-0.032810,0.025390,0.924077,0.983660,0.000477,1.000000,0.994929,0.041197,...,-0.031051,-0.062015,-0.007869,0.008401,-0.079545,0.022406,0.086879,-0.012813,0.007714,0.037104
pitch_count,-0.153254,0.154101,-0.040254,0.032916,0.915949,0.976713,-0.076730,0.994929,1.000000,0.034306,...,-0.042823,-0.053387,-0.003057,0.017797,-0.078721,0.020762,0.092288,-0.008906,0.009939,0.041731
iz_contact_percent,0.449539,-0.162983,-0.386144,0.383450,-0.094503,0.011391,0.049473,0.041197,0.034306,1.000000,...,-0.813359,-0.324193,-0.365757,-0.185842,-0.185682,-0.080031,0.045799,-0.170775,-0.082507,-0.050107


## MinMax Scaler

In [36]:
feature = df.drop(columns = ['last_name',' first_name','player_id','year','xwoba'])
target = df['xwoba']

In [40]:
scaler = MinMaxScaler()
feature = scaler.fit_transform(feature)

In [72]:
X_test = df.drop(columns = ['last_name',' first_name','player_id','year','xwoba'])
y_test = df['xwoba']

In [73]:
X_test = scaler.fit_transform(X_test)

# 데이터를 PICKLE 파일로 저장

CSV파일은 변수의 형식이 모두 초기화 되는 반면,
PKL파일은 변수의 형식도 그대로 남아 있기에 PKL로 저장하였다.

In [81]:
df.to_pickle('./'+'baseball_stat_train.pkl')

In [82]:
test.to_pickle('./'+'baseball_stat_test.pkl')

In [ ]:
train = pd.read_pickle('baseball_stat_train.pkl')
test = pd.read_pickle('baseball_stat_test.pkl')

In [ ]:
train.head(3)

,last_name,first_name,player_id,year,xwoba,oz_swing_percent,oz_swing_miss_percent,oz_contact_percent,out_zone_swing_miss,out_zone_swing,...,fastball_avg_spin,fastball_avg_break,breaking_avg_speed,breaking_avg_spin,breaking_avg_break,offspeed_avg_speed,offspeed_avg_spin,offspeed_avg_break,throws_breaking,throws_offspeed
0,Colon,Bartolo,112526,2016,0.318,30.2,22.4,75.9,91,406,...,2148,17.4,82.5,2282.0,7.9,81.4,1683.0,16.2,1,1
1,Grilli,Jason,276351,2016,0.305,28.4,49.1,50.3,80,163,...,2357,20.5,81.1,2605.0,13.9,84.9,2134.0,19.9,1,1
2,Benoit,Joaquin,276542,2016,0.294,34.7,40.9,57.3,67,164,...,2252,19.1,87.2,2422.0,6.8,84.5,1374.0,13.7,1,1


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4198 entries, 0 to 4197
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   last_name              4198 non-null   object  
 1    first_name            4198 non-null   object  
 2   player_id              4198 non-null   category
 3   year                   4198 non-null   category
 4   xwoba                  4198 non-null   float64 
 5   oz_swing_percent       4198 non-null   float64 
 6   oz_swing_miss_percent  4198 non-null   float64 
 7   oz_contact_percent     4198 non-null   float64 
 8   out_zone_swing_miss    4198 non-null   int64   
 9   out_zone_swing         4198 non-null   int64   
 10  out_zone_percent       4198 non-null   float64 
 11  out_zone               4198 non-null   int64   
 12  iz_contact_percent     4198 non-null   float64 
 13  in_zone_swing_miss     4198 non-null   int64   
 14  in_zone_swing          4198 non-null   i

# AutoML

In [85]:
train = train.drop(columns = ['last_name',' first_name','player_id','year'] )

In [86]:
test_ = test.drop(columns = ['last_name',' first_name','player_id','year'])

## Model Train

In [88]:
setup_rgs = setup(data=train,target='xwoba')

,Description,Value
0,Session id,2732
1,Target,xwoba
2,Target type,Regression
3,Original data shape,"(4198, 27)"
4,Transformed data shape,"(4198, 27)"
5,Transformed train set shape,"(2938, 27)"
6,Transformed test set shape,"(1260, 27)"
7,Ordinal features,2
8,Numeric features,24
9,Categorical features,2


In [89]:
top5 = compare_models(sort='RMSE',n_select=5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lr,Linear Regression,0.0249,0.0011,0.0326,0.4085,0.0246,0.0791,0.6300
ridge,Ridge Regression,0.0249,0.0011,0.0326,0.4101,0.0245,0.0790,0.1400
br,Bayesian Ridge,0.0249,0.0011,0.0326,0.4110,0.0245,0.0790,0.1470
et,Extra Trees Regressor,0.0254,0.0011,0.0333,0.3844,0.0251,0.0810,1.9510
gbr,Gradient Boosting Regressor,0.0253,0.0011,0.0333,0.3855,0.0250,0.0804,1.6040
rf,Random Forest Regressor,0.0255,0.0011,0.0335,0.3773,0.0252,0.0813,4.4290
lightgbm,Light Gradient Boosting Machine,0.0256,0.0011,0.0338,0.3643,0.0255,0.0813,0.4920
ada,AdaBoost Regressor,0.0271,0.0012,0.0350,0.3166,0.0265,0.0877,0.6550
xgboost,Extreme Gradient Boosting,0.0268,0.0013,0.0353,0.3045,0.0266,0.0854,1.2670
en,Elastic Net,0.0318,0.0017,0.0411,0.0654,0.0310,0.1025,0.1410


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

## GridSearch

In [90]:
tuned_top5 = [tune_model(i) for i in top5]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0244,0.0010,0.0321,0.4215,0.0242,0.0775
1,0.0253,0.0012,0.0339,0.4295,0.0253,0.0781
2,0.0235,0.0009,0.0304,0.4480,0.0231,0.0767
3,0.0248,0.0011,0.0329,0.3737,0.0248,0.0794
4,0.0256,0.0011,0.0331,0.4327,0.0250,0.0819
5,0.0262,0.0012,0.0351,0.3644,0.0264,0.0828
6,0.0252,0.0011,0.0328,0.4395,0.0246,0.0783
7,0.0259,0.0011,0.0337,0.4130,0.0254,0.0826
8,0.0255,0.0010,0.0314,0.3234,0.0238,0.0833


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 2 candidates, totalling 20 fits


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0245,0.0010,0.0321,0.4221,0.0242,0.0775
1,0.0253,0.0012,0.0339,0.4299,0.0253,0.0781
2,0.0236,0.0009,0.0304,0.4478,0.0231,0.0767
3,0.0247,0.0011,0.0327,0.3787,0.0247,0.0791
4,0.0257,0.0011,0.0331,0.4327,0.0250,0.0820
5,0.0262,0.0012,0.0351,0.3649,0.0264,0.0828
6,0.0252,0.0011,0.0328,0.4396,0.0246,0.0783
7,0.0259,0.0011,0.0337,0.4132,0.0254,0.0826
8,0.0255,0.0010,0.0314,0.3237,0.0238,0.0833


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

...

Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0259,0.0012,0.0346,0.3269,0.0260,0.0815
1,0.0261,0.0012,0.0348,0.4018,0.0260,0.0807
2,0.0241,0.0010,0.0318,0.3977,0.0241,0.0787
3,0.0251,0.0012,0.0344,0.3157,0.0259,0.0802
4,0.0271,0.0012,0.0353,0.3551,0.0266,0.0868
5,0.0279,0.0014,0.0374,0.2792,0.0282,0.0888
6,0.0267,0.0012,0.0344,0.3830,0.0258,0.0836
7,0.0276,0.0013,0.0362,0.3213,0.0273,0.0883
8,0.0258,0.0011,0.0329,0.2584,0.0250,0.0847


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [91]:
top5

[LinearRegression(n_jobs=-1),
 Ridge(random_state=2732),
 BayesianRidge(),
 ExtraTreesRegressor(n_jobs=-1, random_state=2732),
 GradientBoostingRegressor(random_state=2732)]

## 앙상블

In [92]:
blender_top5 = blend_models(estimator_list = tuned_top5)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0243,0.0010,0.0321,0.4198,0.0242,0.0769
1,0.0253,0.0011,0.0336,0.4401,0.0251,0.0779
2,0.0232,0.0009,0.0299,0.4649,0.0228,0.0757
3,0.0243,0.0011,0.0324,0.3895,0.0245,0.0779
4,0.0257,0.0011,0.0330,0.4361,0.0249,0.0820
5,0.0261,0.0012,0.0349,0.3726,0.0263,0.0828
6,0.0253,0.0011,0.0328,0.4407,0.0245,0.0787
7,0.0258,0.0011,0.0338,0.4095,0.0255,0.0826
8,0.0249,0.0010,0.0309,0.3425,0.0235,0.0816


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

## 최종 모델을 통한 예측

In [93]:
final_model = finalize_model(blender_top5)
prediction = predict_model(final_model, data=test_)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,0.0372,0.0022,0.0464,0.2681,0.0351,0.1199


In [95]:
prediction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255 entries, 0 to 254
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   oz_swing_percent       255 non-null    float32 
 1   oz_swing_miss_percent  255 non-null    float32 
 2   oz_contact_percent     255 non-null    float32 
 3   out_zone_swing_miss    255 non-null    int8    
 4   out_zone_swing         255 non-null    int8    
 5   out_zone_percent       255 non-null    float32 
 6   out_zone               255 non-null    int32   
 7   iz_contact_percent     255 non-null    float32 
 8   in_zone_swing_miss     255 non-null    int8    
 9   in_zone_swing          255 non-null    int32   
 10  in_zone_percent        255 non-null    float32 
 11  in_zone                255 non-null    int32   
 12  edge_percent           255 non-null    float32 
 13  edge                   255 non-null    int32   
 14  whiff_percent          255 non-null    flo

In [96]:
prediction['prediction_label']

0      0.342892
1      0.323907
2      0.342159
3      0.326377
4      0.317184
         ...   
250    0.352746
251    0.311420
252    0.305954
253    0.320448
254    0.331205
Name: prediction_label, Length: 255, dtype: float64

In [110]:
prediction['xwoba']

0      0.369
1      0.350
2      0.391
3      0.362
4      0.318
       ...  
250    0.389
251    0.273
252    0.383
253    0.298
254    0.344
Name: xwoba, Length: 255, dtype: float32

In [99]:
from sklearn.metrics import mean_squared_error

RMSE = mean_squared_error(prediction['xwoba'], prediction['prediction_label'])**0.5
RMSE

0.04639949013558627

In [108]:
player2023 = test[['last_name',' first_name','player_id','xwoba']]

In [111]:
player2023['predicted'] = prediction['prediction_label']

In [112]:
player2023.head(10)

,last_name,first_name,player_id,xwoba,predicted
0,Greinke,Zack,425844,0.369,0.342892
1,Kluber,Corey,446372,0.350,0.323907
2,Hill,Rich,448179,0.391,0.342159
3,Morton,Charlie,450203,0.362,0.326377
4,Scherzer,Max,453286,0.318,0.317184
5,Lynn,Lance,458681,0.367,0.307684
6,Carrasco,Carlos,471911,0.411,0.373870
7,Garcia,Luis,472610,0.311,0.308765
8,Kershaw,Clayton,477132,0.265,0.279061
9,Miley,Wade,489119,0.320,0.342447


이대로 끝내기는 아쉬워서 수비와 운을 배제한 평균자책점인 xERA도 구해보았다.
xERA는 ERA와 마찬가지로 낮을 수록 좋으며, 만약 어떤 투수의 ERA가 xERA보다도 높다면, 운이 없었다거나 팀의 수비가 안좋다는 뜻으로 해석될 수 있다.

In [ ]:
# xERA = lg ERA × (wOBA / lg wOBA)^2                # lg ERA 는 리그 평균 평균자책점. 2023년의 경우 4.35이다.
                                                    

In [115]:
lgwOBA = player2023['predicted'].mean()

In [122]:
player2023['xERA'] = 4.35 * (player2023['predicted'] / lgwOBA ) **2

In [129]:
pd.set_option('display.max_rows', None) ## 모든 열을 출력한다.

In [130]:
player2023.sort_values('xERA')

,last_name,first_name,player_id,xwoba,predicted,xERA
235,Strider,Spencer,675911,0.244,0.232207,2.273715
224,Bednar,David,670280,0.166,0.234082,2.310581
171,McClanahan,Shane,663556,0.263,0.237208,2.372705
55,deGrom,Jacob,594798,0.214,0.251188,2.660616
126,Bautista,Felix,642585,0.236,0.253004,2.699215
65,Romano,Jordan,605447,0.220,0.254301,2.726963
159,Ryan,Joe,657746,0.250,0.266140,2.986784
109,Lopez,Pablo,641154,0.284,0.269090,3.053372
67,Springs,Jeffrey,605488,0.201,0.269836,3.070312
53,German,Domingo,593334,0.349,0.274662,3.181134
